# Remove before submtion for our refrenc only
## Yusuf update 
I completed the code for part one, but there's a huge error in the calculation of the inverse kinematics. I increased the max number of function evaluations to 5000 to give the optimizer more chances to find a solution, but the errors are still quite large.

## The Three Transformation Matrices

T_base:
- Transforms from the "world" coordinate system to our robot's base coordinate system.
- Accounts for where our robot is positioned/oriented in the world.

T_final:
- The transformation we calculate using D‑H parameters.
- Goes from the robot's base frame to the end‑effector frame.
- This is the forward kinematics computes we did in lab 2 part 4.

T_tool:
- Transforms from the end‑effector frame to the actual tool frame.
- Accounts for any gripper attached to the end‑effector.

Overall Transformation:
- The overall transformation from the world frame to the tool frame is given by:
    $$T_{\text{total}} = T_{\text{base}} \; T_{\text{final}} \; T_{\text{tool}}$$
- Since our world frame is the same as the robot base frame then T_base is the identity matrix. 
- So, if we have a tool attached, the overall transformation simplifies to:
  $$T_{\text{total}} = T_{\text{final}} \; T_{\text{tool}}$$
- If no tool is attached, T_tool is also the identity matrix, and $$T_{\text{total}} = T_{\text{final}}$$


In [32]:
import math
import numpy as np
import pandas as pd
from sympy.matrices import Matrix
from scipy.optimize import least_squares
from sympy  import symbols, cos, sin, atan2, pi, lambdify 
from pathlib import Path

In [33]:
#Global variables
q1, q2, q3, q4, q5, q6 = symbols('q1 q2 q3 q4 q5 q6') # Define symbolic variables for joint angles
right = pi/2

- Recursively searches the current working directory (Path.cwd()) for files named `mecharm_control_group_5.csv`.
- Selects the first match whose parent folder path contains the substring `Lab3`.
- If no suitable file is found it raises a FileNotFoundError with instructions on where to place the CSV or how to change the notebook working directory.
- If found, it prints the file path and loads the CSV into a pandas DataFrame named `data`.
- `known_joint_angles` is a NumPy array extracted from the DataFrame containing joint angles J1 to J6.
- `known_position` is a NumPy array extracted from the DataFrame containing end-effector positions X, Y, Z.
- `known_orientation` is a NumPy array extracted from the DataFrame containing end-effector orientations RX, RY, RZ.

In [34]:
for p in Path.cwd().rglob('mecharm_control_group_5.csv'):
    if 'Lab3' in str(p.parent):
        csv_path = p
        break

if csv_path.exists() == False:
    raise FileNotFoundError("mecharm_control_group_5.csv not found. Put it in Lab3/ under the repo or set notebook cwd to the repo root (use %cd).")
else:
    print("Loading CSV from:", csv_path)
    data = pd.read_csv(csv_path)
#Joint Angles from Lab 2 task 3
known_joint_angles = data[['J1','J2','J3','J4','J5','J6']].to_numpy()
#End-Effector Positions from Lab 2 task 3
known_position = data[['X','Y','Z']].to_numpy()
#End-Effector Orientations from Lab 2 task 3
known_orientation = data[['RX','RY','RZ']].to_numpy()

Loading CSV from: /Users/yusufdineh/Documents/GitHub/EE347/Lab3/mecharm_control_group_5.csv


- we are defining our DH table parameters here in a 6x4 matrix called `dh_table`
- DH table rows: [a, alpha, d, offset, offset]

In [35]:
dh_table = [
    [0,         -right,       114,  q1],           # joint 1
    [100,           0,         0,   q2 - right],   # joint 2
    [10,        -right,       0,   q3],           # joint 3
    [0,         right,      96,   q4],           # joint 4
    [0,         -right,       0,   q5],           # joint 5
    [0,         0,         56.5,   q6]              # joint 6
]

- `get_transformation_matrix(a, alpha, d, theta)`
  - Inputs: the four DH parameters for one joint:
  - Returns: a 4×4 SymPy homogeneous transform Matrix that encodes the rotation and translation from frame i to frame i+1 for that DH row.

- `overall_transformation(dh_table)`
  - `T = Matrix(np.identity(4))` initializes the cumulative transform as the 4×4 identity (base frame).  
  - Loop: for each DH row `a, alpha, d, theta` it calls `get_transformation_matrix` to get the per‑joint transform `T_i`, then multiplies `T = T * T_i`. That sequential multiplication composes transforms frame‑by‑frame, so after the loop `T` is the full symbolic transform from the robot base to the end‑effector.  
  - Return: the composed 4×4 SymPy Matrix (use `T[:3,3]` for position and `T[:3,:3]` for rotation when lambdifying).

In [36]:
def get_transformation_matrix(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta), 0, a],
        [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
        [sin(theta)*sin(alpha), cos(alpha)*sin(alpha), cos(alpha), cos(alpha)*d],
        [0, 0, 0, 1]
    ])
def overall_transformation(dh_table):
    T = Matrix(np.identity(4))
    for i in range(len(dh_table)):
        a, alpha, d, theta = dh_table[i]
        T_i = get_transformation_matrix(a, alpha, d, theta)
        T = T * T_i
    return T

- `T_symbolic` Builds a single 4×4 SymPy homogeneous transform from base → end‑effector using your DH table. Result is symbolic (contains q1..q6).
- `fk_num` Create a fast numeric function that maps numeric (q1..q6) → position vector (x,y,z).
- `rot_num` = Create a fast numeric function that maps numeric (q1..q6) → 3×3 rotation matrix.

In [37]:
T_symbolic = overall_transformation(dh_table)
fk_num = lambdify((q1, q2, q3, q4, q5, q6), T_symbolic[:3, 3], modules='numpy')
rot_num = lambdify((q1, q2, q3, q4, q5, q6), T_symbolic[:3, :3], modules='numpy')

`fk` computes end‑effector position [x,y,z] from joint angles using the lambdified forward‑kinematics function fk_num and returns it as a NumPy array.

In [38]:
def fk(q_vals, degrees=True):
    q = np.asarray(q_vals, dtype=float)
    if degrees:
        q = np.radians(q)
    xyz = np.asarray(fk_num(*q), dtype=float).ravel()
    return xyz

# Inverse kinematics helper functions

- `position_error`: compute the error between the forward-kinematics position and the desired target.
- `orientation_error`: compute rotational error (ZYX yaw-pitch-roll) between current forward-kinematics rotation and desired orientation.
  - Helper `_wrap_angle(a)` wraps any angle to [-π, π].
- `link_lengths` computea a upper bound on robot reach from DH rows.

In [39]:
def position_error(q_position, x_target, y_target, z_target):
    pos = fk(q_position)
    return np.array([pos[0] - float(x_target),
                     pos[1] - float(y_target),
                     pos[2] - float(z_target)])

def orientation_error(q_orientation, rx_d, ry_d, rz_d):

    def _wrap_angle(a):
        return (a + np.pi) % (2*np.pi) - np.pi

    q = np.asarray(q_orientation, dtype=float)
    q = np.radians(q)
    rx_d = np.radians(rx_d); ry_d = np.radians(ry_d); rz_d = np.radians(rz_d)

    R = np.array(rot_num(*q), dtype=float)   # 3x3 rotation matrix
    # ZYX (yaw-pitch-roll) extraction prof explained it in class
    roll  = np.arctan2(R[2,1], R[2,2])
    pitch = np.arctan2(-R[2,0], np.sqrt(R[2,1]**2 + R[2,2]**2))
    yaw   = np.arctan2(R[1,0], R[0,0])

    err_roll  = _wrap_angle(roll  - float(rx_d))
    err_pitch = _wrap_angle(pitch - float(ry_d))
    err_yaw   = _wrap_angle(yaw   - float(rz_d))

    return np.array([err_roll, err_pitch, err_yaw])

def link_lengths(dh_table):
    link_lengths = 0.0
    for a, alpha, d, theta in dh_table:
        link_lengths += abs(float(a)) + abs(float(d))
    return link_lengths

- `inverse_kinematics` computes joint angles [q1..q6] to reach desired end-effector pose using numerical optimization.

In [40]:
def inverse_kinematics(x_target, y_target, z_target, rx_d, ry_d, rz_d, q_init, link_lengths, max_iterations=100, tolerance=1e-6):
    distance_from_origin = math.hypot(x_target, y_target, z_target)
    if distance_from_origin > link_lengths:
        raise ValueError("Target outside of range")
    
    def combined_error(q):
        pos_err = position_error(q, x_target, y_target, z_target)
        ori_err = orientation_error(q, rx_d, ry_d, rz_d)
        return np.concatenate((pos_err, ori_err))

    result = least_squares(combined_error, q_init, max_nfev=max_iterations, xtol=tolerance)

    if result.success:
        return result.x
    else:
        raise ValueError("Inverse kinematics did not converge")

## Validation and Accuracy Check

In [44]:
def _angle_diff_deg(a_deg, b_deg):
    # return signed difference a - b mapped to [-180, 180]
    d = (a_deg - b_deg + 180.0) % 360.0 - 180.0
    return d

results = []
max_nfev = 5000
tol = 1e-6
reach = link_lengths(dh_table)
q_init = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

for i, (pos, ori, q_known) in enumerate(zip(known_position, known_orientation, known_joint_angles)):
    x, y, z = pos
    rx, ry, rz = ori
    try:
        q_sol = inverse_kinematics(x, y, z, rx, ry, rz, q_init, reach,
                                   max_iterations=max_nfev, tolerance=tol)
        q_sol = np.asarray(q_sol, dtype=float)  # degrees
        per_joint_signed = np.array([_angle_diff_deg(q_sol[j], q_known[j]) for j in range(6)])
        per_joint_abs = np.abs(per_joint_signed)
        total_error = np.linalg.norm(per_joint_abs)
        status = "ok"
    except Exception as e:
        # mark failure and fill with NaNs
        q_sol = np.full(6, np.nan)
        per_joint_signed = np.full(6, np.nan)
        per_joint_abs = np.full(6, np.nan)
        total_error = np.nan
        status = f"fail: {e}"

    row = {
        "Index": i,
        "X": x, "Y": y, "Z": z,
        "RX": rx, "RY": ry, "RZ": rz,
        # known angles
        "J1_known": q_known[0], "J2_known": q_known[1], "J3_known": q_known[2],
        "J4_known": q_known[3], "J5_known": q_known[4], "J6_known": q_known[5],
        # solved angles
        "J1_sol": float(q_sol[0]) if not np.isnan(q_sol[0]) else np.nan,
        "J2_sol": float(q_sol[1]) if not np.isnan(q_sol[1]) else np.nan,
        "J3_sol": float(q_sol[2]) if not np.isnan(q_sol[2]) else np.nan,
        "J4_sol": float(q_sol[3]) if not np.isnan(q_sol[3]) else np.nan,
        "J5_sol": float(q_sol[4]) if not np.isnan(q_sol[4]) else np.nan,
        "J6_sol": float(q_sol[5]) if not np.isnan(q_sol[5]) else np.nan,
        # per-joint signed and abs errors (deg)
        "J1_err_signed": float(per_joint_signed[0]) if not np.isnan(per_joint_signed[0]) else np.nan,
        "J2_err_signed": float(per_joint_signed[1]) if not np.isnan(per_joint_signed[1]) else np.nan,
        "J3_err_signed": float(per_joint_signed[2]) if not np.isnan(per_joint_signed[2]) else np.nan,
        "J4_err_signed": float(per_joint_signed[3]) if not np.isnan(per_joint_signed[3]) else np.nan,
        "J5_err_signed": float(per_joint_signed[4]) if not np.isnan(per_joint_signed[4]) else np.nan,
        "J6_err_signed": float(per_joint_signed[5]) if not np.isnan(per_joint_signed[5]) else np.nan,
        "J1_err_abs": float(per_joint_abs[0]) if not np.isnan(per_joint_abs[0]) else np.nan,
        "J2_err_abs": float(per_joint_abs[1]) if not np.isnan(per_joint_abs[1]) else np.nan,
        "J3_err_abs": float(per_joint_abs[2]) if not np.isnan(per_joint_abs[2]) else np.nan,
        "J4_err_abs": float(per_joint_abs[3]) if not np.isnan(per_joint_abs[3]) else np.nan,
        "J5_err_abs": float(per_joint_abs[4]) if not np.isnan(per_joint_abs[4]) else np.nan,
        "J6_err_abs": float(per_joint_abs[5]) if not np.isnan(per_joint_abs[5]) else np.nan,
        "Total_error_norm": float(total_error) if not np.isnan(total_error) else np.nan,
        "status": status
    }
    results.append(row)
    print(f"Row {i}: status={status}, total_error={row['Total_error_norm']}")

df_results = pd.DataFrame(results)
out_path = Path("/Users/yusufdineh/Documents/GitHub/EE347/Lab3") / "inverse_kinematics_results.csv"
df_results.to_csv(out_path, index=False)
print("Saved results to", out_path)

Row 0: status=fail: Inverse kinematics did not converge, total_error=nan
Row 1: status=ok, total_error=189.77793696226752
Row 2: status=fail: Inverse kinematics did not converge, total_error=nan
Row 3: status=ok, total_error=173.83521730731812
Row 4: status=ok, total_error=173.87097949595798
Row 5: status=ok, total_error=166.6580645658519
Row 6: status=ok, total_error=254.9610887795845
Row 7: status=ok, total_error=237.62654917396364
Row 8: status=ok, total_error=243.20464435715934
Row 9: status=ok, total_error=274.91120738072107
Row 10: status=ok, total_error=221.43044356967582
Row 11: status=ok, total_error=228.1562227772985
Row 12: status=ok, total_error=157.8226134402234
Row 13: status=ok, total_error=157.67731990179794
Saved results to /Users/yusufdineh/Documents/GitHub/EE347/Lab3/inverse_kinematics_results.csv
